<a href="https://colab.research.google.com/github/nebojsa55/Computational-Genomics_MidTerm-Project/blob/master/notebooks/3.%20Linear-regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Description
We postulated that this data set is probably suitable for **linear regression** in the previous notebook. This statement will be exploited in this notebook to see if the rmse can be below **5.9**, what was our best score with RFR.


In [1]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

# Navigate to the folder cointaing our data
%cd 'drive/MyDrive/ETF/Master/Computational-Genomics/Project/data'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/ETF/Master/Computational-Genomics/Project/data


In [2]:
anno = pd.read_csv('anoSC1_v11_nokey.csv', delimiter = ',', index_col = 0)
HTA20_RMA = pd.read_csv('HTA20_RMA.csv', delimiter = ',', index_col = 0).transpose()

# Sync the X and y data by sorting the labels

df1 = anno.sort_index()
df2 = HTA20_RMA.sort_index()

X = df2.iloc[np.array(np.logical_not(df1['GA'].isna())),:]
y = df1.dropna().loc[:,['GA','Batch']]

# Check to see if the indexes are the same
(X.index == y.index).all()

True

In [3]:
# Drop Sample_X samples

X = X.iloc[32:,:]
y = y.iloc[32:,:]

In [4]:
from sklearn.preprocessing import StandardScaler

XX = np.zeros(X.shape)
for i in [1,2,3,4,5,6,7,8,9,10]:
    scale = StandardScaler()
    indices = np.bool8(y['Batch'] == i)
    Xtemp = X.iloc[indices,:]
    scale.fit(Xtemp)
    XX[indices,:] = scale.transform(Xtemp)

# delete batch column
yy = y['GA']

In [11]:
from sklearn.feature_selection import f_regression,SelectKBest
from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error,make_scorer
from sklearn.model_selection import KFold
import warnings 

# Create the scorer for cross_val_score
scorer = make_scorer(mean_squared_error, greater_is_better = False)

# Create KFold 
kfold = KFold(n_splits = 10, shuffle = True, random_state = 42)

# Create our linear model
lin_model = ElasticNetCV(cv = kfold, random_state = 42)

# Create lists where cv score will be saved
cv_lin = []

# Ignore convergence warnings
warnings.filterwarnings("ignore")

for ktemp in [200, 500, 1000]:
  X_best = SelectKBest(f_regression,k = ktemp).fit_transform(XX,yy)
  
  # Elastic Net
  cv_lin.append(cross_val_score(lin_model, 
                          X_best, 
                          yy,
                          cv = kfold, 
                          scoring = scorer, 
                          verbose = 3))
  

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  ................................................................
[CV] .................................. , score=-24.510, total=  13.1s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   13.1s remaining:    0.0s


[CV] .................................. , score=-45.612, total=  11.8s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   25.0s remaining:    0.0s


...

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.2min remaining:    0.0s


[CV] .................................. , score=-42.744, total= 1.4min
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.6min remaining:    0.0s


[CV] .................................. , score=-20.199, total= 1.4min
[CV]  ................................................................
[CV] .................................. , score=-21.516, total= 1.4min
[CV]  ................................................................
[CV] .................................. , score=-32.073, total= 1.2min
[CV]  ................................................................
[CV] .................................. , score=-21.660, total= 1.4min
[CV]  ................................................................
[CV] .................................. , score=-17.677, total= 1.3min
[CV]  ................................................................
[CV] .................................. , score=-37.821, total= 1.5min
[CV]  ................................................................
[CV] .................................. , score=-15.047, total= 1.4min
[CV]  ................................................................
[CV] .

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed: 13.5min finished


In [12]:
rmse_lin = [np.mean(np.sqrt(np.abs(x))) for x in cv_lin]
rmse_lin

[5.340163197515788, 4.928363149626769, 4.956463607432446]

## Results

This score is lower than the score of a previous model by one whole point! With this, our analysis is finished, and the conclusion is that **Elastic Net linear regressor** is the best suited for this kind of problem. **500** best features were selected.

The result of **4.92836** would be good enough for the 19th best result at the challenge, where the RMSE of **4.5442** was the best. 

However, the obtained error should be taken with a grain of salt (or rather a mountain) since we had only **30-40** samples for testing, whereas the challengers had to test **10 times** more. In those conditions, I believe that the model would most certainly produce a worse result than 4.928, but I think that the error would be in the range of 5 to 6.

## Conclusion

* The best model for this challenge was linear model
* There were outliers in the set **'GSE113966'** 